# **Data Dump into PostgreSQL**

In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
import urllib.parse # Import the urllib.parse module

# Database connection parameters
db_host = '34.50.87.186'
db_name = 'db_ftde01'
db_user = 'ftde01'
db_password = 'ftde01!@#' # This password contains a special character '@' that needs to be URL encoded.
db_port = '5432'

# List of CSV file paths
csv_files = [
    '/data_management_payroll.csv',
    '/data_performance_management.csv',
    '/data_training_development.csv'
]

# Connect to PostgreSQL, URL encode the password to handle special characters
engine = create_engine(f'postgresql+psycopg2://{db_user}:{urllib.parse.quote_plus(db_password)}@{db_host}:{db_port}/{db_name}')
conn = engine.raw_connection()
cursor = conn.cursor()

for csv_file_path in csv_files:
    # Extract table name from file name
    table_name = os.path.splitext(os.path.basename(csv_file_path))[0]

    # Read CSV into DataFrame and convert column names to lowercase
    df = pd.read_csv(csv_file_path)
    df.columns = df.columns.str.lower() # Convert column names to lowercase

    # Generate CREATE TABLE statement with quoted column names
    create_table_query = f'CREATE TABLE IF NOT EXISTS {table_name} ('

    for col, dtype in zip(df.columns, df.dtypes):
        if 'int' in str(dtype):
            sql_dtype = 'INTEGER'
        elif 'float' in str(dtype):
            sql_dtype = 'FLOAT'
        elif 'bool' in str(dtype):
            sql_dtype = 'BOOLEAN'
        else:
            sql_dtype = 'TEXT'
        create_table_query += f'"{col}" {sql_dtype}, ' # Quote column names

    create_table_query = create_table_query.rstrip(', ') + ');'

    # Execute CREATE TABLE statement
    cursor.execute(create_table_query)
    conn.commit()

    # Insert DataFrame into PostgreSQL
    df.to_sql(table_name, engine, if_exists='append', index=False)

# Close the connection
cursor.close()
conn.close()

Table data_management_payroll created and data from /data_management_payroll.csv inserted successfully.
Table data_performance_management created and data from /data_performance_management.csv inserted successfully.
Table data_training_development created and data from /data_training_development.csv inserted successfully.


# **Data Dump into MongoDB**

In [ ]:
from pymongo import MongoClient

# Replace the URI string with your MongoDB deployment's connection string.
client = MongoClient("mongodb://localhost:27017/")

# Access the database
db = client['dskola']

import pandas as pd

# Read the CSV file into a DataFrame
csv_file_path = '/data_recruitment_selection.csv'
df = pd.read_csv(csv_file_path)

# Convert DataFrame to dictionary
data = df.to_dict(orient='records')

# Get the collection
collection_name = 'data_recruitment_selection'
collection = db[collection_name]

# Insert data into the collection
collection.insert_many(data)

# **ETL from Database (OLTP Structured) into Data Warehouse (OLAP)**

In [ ]:
import psycopg2
import psycopg2.extras

def extract_and_load_tables(source_conn_str, target_conn_str, tables):
    source_cursor = None  # Initialize source_cursor to None
    target_cursor = None  # Initialize target_cursor to None
    source_conn = None  # Initialize source_conn to None
    target_conn = None  # Initialize target_conn to None
    try:
        # Connect to source and target databases
        source_conn = psycopg2.connect(source_conn_str)
        target_conn = psycopg2.connect(target_conn_str)

        source_cursor = source_conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        target_cursor = target_conn.cursor()

        for table_name in tables:
            # Fetch column names and data from source table
            source_cursor.execute(f"SELECT * FROM {table_name}")
            rows = source_cursor.fetchall()

            if not rows:
                print(f"No data found in table {table_name}. Skipping...")
                continue

            # Create table in target database if not exists
            create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} (LIKE {table_name} INCLUDING ALL)"
            target_cursor.execute(create_table_query)

            # Construct INSERT query dynamically based on column names
            columns = ','.join(rows[0].keys())  # Assuming the first row has all column names
            insert_query = f"INSERT INTO {table_name} ({columns}) VALUES %s"

            # Prepare data for insertion
            data = [tuple(row.values()) for row in rows]

            # Execute the INSERT query
            psycopg2.extras.execute_values(target_cursor, insert_query, data)

        # Commit changes to the target database
        target_conn.commit()

    except psycopg2.Error as e:
        print(f"Error: {e}")

    finally:
        # Check if variables are not None before closing
        if source_cursor is not None:
            source_cursor.close()
        if target_cursor is not None:
            target_cursor.close()
        if source_conn is not None:
            source_conn.close()
        if target_conn is not None:
            target_conn.close()

if __name__ == "__main__":
    source_conn_str = "dbname=db_ftde01 user=ftde01 password=ftde01!@#"
    target_conn_str = "dbname=data_warehouse user=postgres password=gofootball"
    tables_to_extract = ["data_management_payroll", "data_performance_management", "data_training_development"]

    extract_and_load_tables(source_conn_str, target_conn_str, tables_to_extract)

# **ETL from Database (OLTP Unstructured) into Data Warehouse (OLAP)**

In [ ]:
import psycopg2
from psycopg2 import sql
from datetime import datetime

import pymongo

# MongoDB connection string
mongo_uri = "mongodb://localhost:27017/"
client = pymongo.MongoClient(mongo_uri)

# Database name and collection name
mongo_db_name = 'dskola'
mongo_collection_name = 'data_recruitment_selection'

# Connect to the database and collection
db = client[mongo_db_name]
collection = db[mongo_collection_name]

# Fetch all documents from the collection
mongo_documents = collection.find()

# PostgreSQL connection parameters
postgres_host = 'localhost'
postgres_port = '5432'
postgres_db = 'data_warehouse'
postgres_user = 'postgres'
postgres_password = 'gofootball'

# Connect to PostgreSQL
conn = psycopg2.connect(
    host=postgres_host,
    port=postgres_port,
    database=postgres_db,
    user=postgres_user,
    password=postgres_password
)
cursor = conn.cursor()

# Truncate table before inserting (optional, to ensure clean insert)
truncate_table_sql = "TRUNCATE TABLE candidates;"
cursor.execute(truncate_table_sql)
conn.commit()

# Insert data into PostgreSQL table
for doc in mongo_documents:
    insert_sql = """
        INSERT INTO candidates (candidate_id, name, gender, age, position, application_date, status, interview_date, offer_status)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    cursor.execute(insert_sql, (
        doc.get('CandidateID'),
        doc.get('Name'),
        doc.get('Gender'),
        doc.get('Age'),
        doc.get('Position'),
        doc.get('ApplicationDate').replace(tzinfo=None) if 'ApplicationDate' in doc else None,
        doc.get('Status'),
        doc.get('InterviewDate').replace(tzinfo=None) if 'InterviewDate' in doc else None,
        doc.get('OfferStatus')
    ))

# Commit the transaction
conn.commit()

# Close cursor and connection
cursor.close()
conn.close()

# **ETL from Data Warehouse (OLAP) into Data Mart (Google Spreadsheets)**

In [ ]:
import psycopg2
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Database credentials
DB_HOST = 'localhost'
DB_NAME = 'data_warehouse'
DB_USER = 'postgres'
DB_PASS = 'gofootball'
DB_PORT = '5432'

# Google Sheets credentials file
CREDENTIALS_FILE = '/client_secret_571893854046-ehgmu6jogk5gavhfuso0ftlg935vqrfg.apps.googleusercontent.com.json'
SHEET_NAME = 'Project 4 - Data Mart'

# List of table names to extract
TABLES = ['employee_demographics', 'applicant_demographics', 'human_resources_costs', 'employee_performances']

# Extract data from PostgreSQL
def extract_data():
    conn = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASS,
        port=DB_PORT
    )
    data_frames = {}
    for table in TABLES:
        query = f"SELECT * FROM {table}"
        data_frames[table] = pd.read_sql(query, conn)
    conn.close()
    return data_frames

# Load data into Google Sheets
def load_data(data_frames):
    scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, scope)
    client = gspread.authorize(credentials)

    # Open the Google Sheet
    sheet = client.open(SHEET_NAME)

    # Clear existing content
    for table in TABLES:
        worksheet = sheet.worksheet(table)
        worksheet.clear()

    # Load each table into a separate sheet
    for table, df in data_frames.items():
        worksheet = sheet.worksheet(table)
        worksheet.update([df.columns.values.tolist()] + df.values.tolist())

# Main function to execute ETL process
def main():
    data_frames = extract_data()
    load_data(data_frames)

if __name__ == "__main__":
    main()